In [2]:
from networkx.generators.random_graphs import random_regular_graph
import networkx as nx
import numpy as np
from scipy.special import expit
from scipy.optimize import minimize
from Unit import unit
from sklearn.linear_model import LogisticRegression
import pickle
import copy

In [3]:
with open('nodeList.pkl', 'rb') as fname:
    nodeList = pickle.load(fname)
with open('graph.pkl', 'rb') as fname:
    graph = pickle.load(fname)

In [4]:
As = np.zeros(len(nodeList))
Cs = np.zeros((len(nodeList), 3))
Ms = np.zeros(len(nodeList))
Ys = np.zeros(len(nodeList))

for node in nodeList:
    As[node] = nodeList[node].A
    Cs[node] = nodeList[node].C
    Ms[node] = nodeList[node].M
    Ys[node] = nodeList[node].Y
    
clfA = LogisticRegression()
clfA.fit(Cs, As)
tauA = {'intcp': clfA.intercept_[0], 'C0': clfA.coef_[0][0], 'C1': clfA.coef_[0][1], 'C2': clfA.coef_[0][2]}

cov = np.concatenate((Cs, np.reshape(As, (As.shape[0], 1)), np.reshape(Ms, (Ms.shape[0], 1))), axis=1)
clfY = LogisticRegression()
clfY.fit(cov, Ys)
tauY = {'intcp': clfY.intercept_[0], 'C0': clfY.coef_[0][0], 'C1': clfY.coef_[0][1], 'C2': clfY.coef_[0][2],
         'A': clfY.coef_[0][3], 'M': clfY.coef_[0][4]}

In [5]:
tauM = np.random.rand(7)

In [6]:
def pseudoLikelihoodM(tauM, nodeList):
    total = 0
    for i in nodeList:
        sumM = tauM[0] + (tauM[1] * nodeList[i].C[0]) + (tauM[2] * nodeList[i].C[1])
        sumM += (tauM[3] * nodeList[i].C[2]) + (tauM[4] * nodeList[i].A)

        for nbor in nodeList[i].adj:
            sumM += (tauM[5] * nodeList[nbor].A) + (tauM[6] * nodeList[nbor].M)
        
        if nodeList[i].M == 1:
            total += np.log(expit(sumM))
        else:
            total += np.log(1 - expit(sumM))
    
    return (-1 * total)

In [7]:
resultM = minimize(pseudoLikelihoodM, tauM, (nodeList))